## Building a First Data Set for Baselinemodel

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import re
from fuzzywuzzy import fuzz
import glob

In [11]:
# Initialize Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def extract_pose_features_MP(image_path):
    image = cv2.imread(image_path)

    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2, # BlazePose GHUM Heavy, BlazePose GHUM Full, BlazePose GHUM Lite (flags 0,1,2)
        min_detection_confidence=0.5) as pose:

        # Convert the image to RGB before processing
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            pose_features = []
            for landmark in results.pose_landmarks.landmark:
                pose_features.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            return np.array(pose_features)
        else:
            return None



def extract_class(filename):
    match = re.search(r'_ *(\d+)\.', filename)
    if match:
        return int(match.group(1))  # Convert to int to remove leading zeros
    else:
        return 42 

In [225]:
#path_to_images = "/Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/pictures/lateral/"
path_to_images = "/Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/ExtractedFrames/"
skier_images = glob.glob(path_to_images + "*.jpg") + glob.glob(path_to_images + "*.png")

path_to_videos = "/Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/videos/lateral/"
skier_videos = glob.glob(path_to_videos + "*.[mM][pP]4") + glob.glob(path_to_videos + "*.[mM][oO][vV]")
skier_videos = [video for video in skier_videos if "_lq" not in video] # exclude all videos containing "_lq" to avoid double videos


### Extracting Features using MediaPipe 

In [ ]:
X_df = pd.DataFrame()

for image in skier_images:
    one_row_df = pd.DataFrame(extract_pose_features_MP(image)).T
    one_row_df["file_names"] = os.path.basename(image)
    X_df = pd.concat([X_df, one_row_df])



I0000 00:00:1734620880.053953 8936762 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-20.7.2), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
W0000 00:00:1734620880.584113 9108213 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734620881.099566 9108210 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1734620881.326521 8936762 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-20.7.2), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
W0000 00:00:1734620881.612220 9108227 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734620881.803856 9108227 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

### Featureextraction with YOLO

In [297]:
from ultralytics import YOLO

# Load a pretrained YOLO11n-pose Pose model
model = YOLO("yolo11n-pose.pt")

def extract_pose_features_YOLO(image_path_list):

    X_df = pd.DataFrame()

    for image in image_path_list:
        results = model(image)
        
        result_data_tensor = results[0].keypoints.data

        # Convert tensor to list
        list_data = result_data_tensor.tolist()
        flat_list = result_data_tensor.flatten().tolist()

        one_row_df = pd.DataFrame(flat_list).T
        one_row_df["file_names"] = os.path.basename(image)
        if len(flat_list) < 60:
            X_df = pd.concat([X_df, one_row_df])

    return X_df


In [298]:
YOLO_X_df = extract_pose_features_YOLO(skier_images)
YOLO_X_df


image 1/1 /Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/ExtractedFrames/Klauser Hannes_004.jpg: 384x640 1 person, 243.8ms
Speed: 7.2ms preprocess, 243.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/ExtractedFrames/Klauser Hannes_005.jpg: 384x640 1 person, 225.4ms
Speed: 3.7ms preprocess, 225.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/ExtractedFrames/Klauser Hannes_001.jpg: 384x640 1 person, 226.8ms
Speed: 3.7ms preprocess, 226.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 armswing/ExtractedFrames/Klauser Hannes_002.jpg: 384x640 1 person, 219.0ms
Speed: 4.3ms preprocess, 219.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Volumes/ThesisUSB/AVP XC Skiing Data/Skating 2-1 arm

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,file_names
0,1022.843811,384.223328,0.711301,0.000000,0.000000,0.116826,1017.894470,370.785278,0.787073,0.0,...,830.130066,721.418335,0.996470,910.988159,861.487976,0.953584,723.496460,877.292542,0.981310,Klauser Hannes_004.jpg
0,1067.463623,207.968048,0.934054,0.000000,0.000000,0.451151,1053.427490,195.838531,0.940903,0.0,...,1011.097107,633.013733,0.995018,918.377075,753.535950,0.939372,950.525085,782.480896,0.967052,Klauser Hannes_005.jpg
0,964.492859,290.223511,0.874690,0.000000,0.000000,0.220242,955.208130,279.495667,0.910486,0.0,...,887.744812,684.628601,0.997421,926.097900,816.353394,0.959675,837.961853,839.362061,0.980185,Klauser Hannes_001.jpg
0,981.484741,319.100616,0.890381,0.000000,0.000000,0.241516,972.041077,307.293304,0.925547,0.0,...,894.614685,703.922607,0.998237,883.747559,842.620972,0.967808,826.920654,863.472290,0.985631,Klauser Hannes_002.jpg
0,1006.329224,364.574615,0.839183,0.000000,0.000000,0.205378,998.116211,350.749512,0.893656,0.0,...,886.781311,710.076416,0.997968,921.355957,844.316528,0.979274,794.906433,871.336853,0.990644,Klauser Hannes_003.jpg
0,695.680969,454.930023,0.814296,0.000000,0.000000,0.225221,690.293335,443.915588,0.858147,0.0,...,661.030273,735.134583,0.994608,542.638611,838.583679,0.950629,647.193054,848.055725,0.975985,Niemeyer Felix _005.jpg
0,931.705322,251.558487,0.833656,0.000000,0.000000,0.310320,925.719177,242.334915,0.905524,0.0,...,840.333069,487.343994,0.991045,834.122192,585.123474,0.952023,808.192932,596.224548,0.977304,Niemeyer Felix _004.jpg
0,915.125061,290.616791,0.777927,0.000000,0.000000,0.227126,912.911377,280.701935,0.829270,0.0,...,847.912231,466.652374,0.990732,817.777649,552.696777,0.920893,817.694885,570.341919,0.962639,Niemeyer Felix _003.jpg
0,847.770142,381.206635,0.726299,0.000000,0.000000,0.187282,843.355042,372.255280,0.762729,0.0,...,747.523132,592.317627,0.994477,724.720947,679.534058,0.954025,722.078613,694.840820,0.975212,Niemeyer Felix _002.jpg
0,775.951355,376.589539,0.863851,0.000000,0.000000,0.290458,773.394165,366.517853,0.869167,0.0,...,667.943787,607.051941,0.998465,693.752747,716.751953,0.984346,624.525269,713.461731,0.991213,Niemeyer Felix _001.jpg


### Extract Class (label) from Filename

<div class="alert alert-block alert-info"> 
<b>Assumption:</b> The Numbers in each filename are correctly assigned to each class
</div>

In [299]:
image_file_names = []
video_file_names = []

for image_path in skier_images:
    file_name_with_extension = os.path.basename(image_path)
    image_file_names.append(file_name_with_extension)

for video_path in skier_videos:
    file_name_with_extension = os.path.basename(video_path)
    video_file_names.append(file_name_with_extension)


videos_and_images_dict = {}

for video_string in video_file_names:
    images_per_vid_dict = {}
    for image_string in image_file_names:
        score = fuzz.ratio(video_string, image_string) # fuzzywuzzy used to compare simmilarity of strings
        images_per_vid_dict[image_string] = score

    # build from 5 images with hightest matching score
    top_5 = dict(sorted(images_per_vid_dict.items(), key=lambda item: item[1], reverse=True)[:5])
    key_list = list(top_5.keys())

    videos_and_images_dict[video_string] = key_list

In [300]:
file_names = []
for key in videos_and_images_dict:
    file_names = file_names + videos_and_images_dict[key] 

In [ ]:
y_df = pd.DataFrame(file_names, columns=["file_names"])
y_df['class'] = y_df['file_names'].apply(extract_class) 

### Feature Label Matrix Merge

In [302]:
df_merged = YOLO_X_df.merge(y_df, on='file_names')
df_merged = df_merged.drop("file_names", axis=1)
df_merged

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,class
0,1022.843811,384.223328,0.711301,0.0,0.0,0.116826,1017.894470,370.785278,0.787073,0.0,...,830.130066,721.418335,0.996470,910.988159,861.487976,0.953584,723.496460,877.292542,0.981310,4
1,1067.463623,207.968048,0.934054,0.0,0.0,0.451151,1053.427490,195.838531,0.940903,0.0,...,1011.097107,633.013733,0.995018,918.377075,753.535950,0.939372,950.525085,782.480896,0.967052,5
2,964.492859,290.223511,0.874690,0.0,0.0,0.220242,955.208130,279.495667,0.910486,0.0,...,887.744812,684.628601,0.997421,926.097900,816.353394,0.959675,837.961853,839.362061,0.980185,1
3,981.484741,319.100616,0.890381,0.0,0.0,0.241516,972.041077,307.293304,0.925547,0.0,...,894.614685,703.922607,0.998237,883.747559,842.620972,0.967808,826.920654,863.472290,0.985631,2
4,1006.329224,364.574615,0.839183,0.0,0.0,0.205378,998.116211,350.749512,0.893656,0.0,...,886.781311,710.076416,0.997968,921.355957,844.316528,0.979274,794.906433,871.336853,0.990644,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.000000,0.000000,0.497040,0.0,0.0,0.191782,1017.193176,435.659546,0.523907,0.0,...,992.283020,516.229858,0.963502,981.713745,547.624390,0.885108,977.409668,550.557617,0.928520,2
66,2911.909668,1206.167603,0.762123,0.0,0.0,0.491739,2907.693604,1197.320312,0.728565,0.0,...,2838.177246,1371.287476,0.988937,2834.304688,1434.179688,0.967561,2797.919678,1443.246826,0.974191,5
67,2911.909668,1206.167603,0.762123,0.0,0.0,0.491739,2907.693604,1197.320312,0.728565,0.0,...,2838.177246,1371.287476,0.988937,2834.304688,1434.179688,0.967561,2797.919678,1443.246826,0.974191,5
68,0.000000,0.000000,0.165736,0.0,0.0,0.037190,0.000000,0.000000,0.175642,0.0,...,1238.181152,1380.198608,0.995035,1133.544434,1440.423096,0.966563,1193.099609,1435.884888,0.986433,3


### ModelTraining

In [112]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [314]:
X_train, X_test, y_train, y_test = train_test_split(df_merged.iloc[:, : 51], df_merged["class"], test_size=0.2, shuffle=True, random_state=42)

In [316]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")


X_train shape: (56, 51)
y_train shape: (56,)


In [317]:
scaler = StandardScaler()

In [318]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.5714285714285714

In [319]:
#Definieren der Parameter
gb_clf = GradientBoostingClassifier(n_estimators=10, max_depth=8, learning_rate=0.1, random_state=0)

#Pipeline definieren
gb_pipe = make_pipeline(scaler, gb_clf)

#"Aufruf" der Pipeline
gb_pipe.fit(X_train, y_train)

#Vorhersage machen
y_train_pred_gb = gb_pipe.predict(X_train)
y_test_pred_gb = gb_pipe.predict(X_test)

#accuracy ermitteln - Evaluation
print(f"accuracy_score Train: {round(accuracy_score(y_train, y_train_pred_gb),5)}")
print(f"accuracy_score Test: {round(accuracy_score(y_test, y_test_pred_gb),5)}")


accuracy_score Train: 1.0
accuracy_score Test: 0.5


In [320]:
from sklearn.ensemble import RandomForestClassifier
#Definieren der Parameter
rf_clf = RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=0, n_jobs=-1)

#Pipeline definieren
rf_pipe = make_pipeline(scaler, rf_clf)

#"Aufruf" der Pipeline
rf_pipe.fit(X_train, y_train)

#Vorhersage machen
y_train_pred_rf = rf_pipe.predict(X_train)
y_test_pred_rf = rf_pipe.predict(X_test)

#accuracy ermitteln - Evaluation
print(f"accuracy_score Train: {round(accuracy_score(y_train, y_train_pred_rf),5)}")
print(f"accuracy_score Test: {round(accuracy_score(y_test, y_test_pred_rf),5)}")

accuracy_score Train: 1.0
accuracy_score Test: 0.28571
